In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

from tqdm.auto import tqdm

In [2]:
from darts import TimeSeries
from darts.metrics import mape, mase, rmse, mae, smape, mse

from darts.utils.missing_values import fill_missing_values

## Prepare Dataset

In [3]:
#Default settings for experiment
arg_model = "tsrnn" #Options: 'trfbb', 'tsrnn', 'trfbf'
arg_dset = "lsm" #Datasets -- Spain: 'ree', AEP, DAYTON: 'dyt' London: 'lsm'

attr_dset_smpl_rt = 24 if arg_dset == "aep" else (48 if arg_dset == "lsm" else 24) #Samples per day. Spain, AEP: 24, London: 48
# param_dset_lookback_weeks = 5
# param_dset_forecast = 48 if arg_dset == "lsm" else 24
param_dset_lookback_weeks = 9
param_dset_forecast = 168 if arg_dset == "lsm" else 84 # 3.5days = 168
param_dset_train_stride = 48 #Choose a coprime value to the forecast so all reading frames are eventually considered
param_dset_test_stride = 'same' #tsrnn paper uses 1 week
param_dset_lookback = param_dset_lookback_weeks*7*attr_dset_smpl_rt - param_dset_forecast

param_trf_weather = False



In [4]:
import os
os.system('')
import pgzip
import pickle
import copy
import torch
import datetime

# path='Datasets/LondonSmartMeter' # if under experiments folder
path = './LondonSmartMeter'
seq_len = param_dset_lookback
pred_horz = param_dset_forecast
weather = param_trf_weather
timestamps = False
weather = False


if 'lsm_dict.pkl.pgz' not in os.listdir(path):
    subprocess.check_call('python ./LondonSmartMeter_hhour.py ./LondonSmartMeter lsm_dict.pkl')
if 'londonWeather.pkl.pgz' not in os.listdir(path):   
    raise ValueError

with pgzip.open(os.path.join(path,'lsm_dict.pkl.pgz'),'rb') as f:
    s_dict = pickle.load(f)
    


In [5]:
self_has_weather = weather
self_return_timestamps = timestamps

self_weather_dict = None
if self_has_weather:
    with pgzip.open(os.path.join(path,'londonWeather.pkl.pgz'),'rb') as f2:
        weather_dict = pickle.load(f2)

    self_weather_dict = weather_dict

#s_dict is dictionary as follows: { lclid: (start_timestamp,Tensor), ...}

#List to store the dataset indices corresponding to each household
self_household_idxs = [None]*len(s_dict)
#List to hold the split series
self_series = [None]*len(s_dict)
#These two lists will be converted to torch tensors
self_start_times = []
self_pred_starttimes = []
#These two lists stores the datetime format
self_start_times__ = []
#self.pred_starttimes__ = []

index_count = 0

# Enumerate all keys in dictionary -> lclid
for index, lclid in enumerate(s_dict.keys()):    
    start_time, s_tensor = s_dict[lclid]
    #start_time[0] = start_time[0]%400
    
    #pad_amt = (seq_len+pred_horz) - (len(s_tensor)%(seq_len+pred_horz))
    #s_tensor = torch.nn.functional.pad(s_tensor,pad = (pad_amt,0), value = torch.nan)
    
    #Split into chunks of seq_len+pred_horz length each
    s_tensors = s_tensor.split(seq_len+pred_horz)

    #Compute start timestamps for splits
    start_times = [None]*len(s_tensors)
    pred_starttimes = [None]*len(s_tensors)

    for i in range(len(start_times)):
        minutes_delta = 30*i*(seq_len + pred_horz)
        time_delta = datetime.timedelta(minutes=minutes_delta)
        new_start_time = start_time + time_delta
        start_times[i] = new_start_time

    # Reinitialised for computation of prediction start time
    minutes_delta = 30*seq_len
    time_delta = datetime.timedelta(minutes=minutes_delta)

    #Compute prediction start timestamps
    for i in range(len(pred_starttimes)):
        new_predtime = start_times[i] + time_delta
        pred_starttimes[i] = [new_predtime.year,
                              new_predtime.month,
                              new_predtime.day,
                              new_predtime.hour,
                              new_predtime.minute,
                              new_predtime.second]

    ''' 
    A shallow copy, li2, is created using copy.copy(), 
    preserving the top-level structure but sharing references to the inner lists. 
    A deep copy, li3, is created using copy.deepcopy(), resulting in a completely 
    independent copy of li1, including all nested elements
    '''
    
    start_times__ = copy.deepcopy(start_times)
    for i in range(len(start_times)):
        new_start_time = start_times[i]
        start_times[i] = [new_start_time.year,
                          new_start_time.month,
                          new_start_time.day,
                          new_start_time.hour,
                          new_start_time.minute,
                          new_start_time.second]

    #Remove last if length less than the others
    if s_tensors[-1].shape[0] < seq_len+pred_horz:
        s_tensors = s_tensors[:-1]
        start_times = start_times[:-1]
        pred_starttimes = pred_starttimes[:-1]
        start_times__ = start_times__[:-1]

    if len(s_tensors) == 0:
        self_series[index] = torch.empty(0)                    
    else:
        '''
        torch.stack()
        Concatenates a sequence of tensors along a new dimension.
        All tensors need to be of the same size.
        '''
        s_tensors = torch.stack(s_tensors)
        # Remove invalid (more than 4/5 (80%) of series is 0 or nan)
        s_tensors[s_tensors==0] = torch.nan # 0s are invalid too, replace with nan
        # Count those tensor index with less than 80% invalid
        sel = (( (s_tensors==0) | s_tensors.isnan()).sum(dim=-1) < (4*(seq_len+pred_horz)//5))
        s_tensors = s_tensors[sel]
        
        # Get start, prediction times for those with less than 80% nan
        start_times_ = [start_times[i] for i in range(len(start_times)) if sel[i]]
        pred_starttimes_ = [pred_starttimes[i] for i in range(len(pred_starttimes)) if sel[i]]
        _start_times__ = [ start_times__[i] for i in range(len(start_times__)) if sel[i]]
        
        self_series[index] = s_tensors
        if index==0:
            print("before series:", self_series[0].shape)
        self_start_times = self_start_times + start_times_
        if index==0:
            print("start_times_ after: ", start_times_)
            print("start_times after: ", start_times)
        self_pred_starttimes = self_pred_starttimes + pred_starttimes_
        self_start_times__ = self_start_times__ + _start_times__

    if len(s_tensors) == 0:
        self_household_idxs[index] = []
    else:
        self_household_idxs[index] = list(range(index_count,index_count+len(s_tensors)))
        index_count = index_count + len(s_tensors)
        
print("before concat series:", len(self_series))
print("before concat series:", self_series[0].shape)
# Concat all households series but retain 1680 seq_len predictions
self_series = torch.cat(self_series,dim=0).unsqueeze(-1)
print("before normalization series:", self_series.shape)

self_start_times = torch.tensor(self_start_times,dtype = torch.long)
self_pred_starttimes = torch.tensor(self_pred_starttimes,dtype = torch.long)

#self.series[:,:seq_len] = self.series[:,:seq_len].nan_to_num(nan=0.,posinf=0.,neginf=0.)

#Series normalization
smin = self_series.nan_to_num(nan=torch.finfo(self_series.dtype).max).amin(dim=-2,keepdim=True)
smax = self_series.nan_to_num(nan=torch.finfo(self_series.dtype).min).amax(dim=-2,keepdim=True)
# Normalize over dimension -2, which is seq_len + prediction len
self_series = (self_series - smin.broadcast_to(self_series.shape))/(smax-smin+1e-10).broadcast_to(self_series.shape)

print("total series shape: ", self_series.shape)


if self_has_weather:
    #Weather series normalization
    #Only dimensions 0, 3, 4, 5, 6, 7 needs normalization
    wdmin = self_weather_dict['tensor'].nan_to_num(nan=torch.finfo(self_weather_dict['tensor'].dtype).max).amin(dim=-2,keepdim=True)
    wdmax = self_weather_dict['tensor'].nan_to_num(nan=torch.finfo(self_weather_dict['tensor'].dtype).min).amax(dim=-2,keepdim=True)
    self_weather_dict['tensor'] = (self_weather_dict['tensor'] - wdmin.broadcast_to(self_weather_dict['tensor'].shape))\
        /(wdmax-wdmin + 1e-10).broadcast_to(self_weather_dict['tensor'].shape)

    self_weather_dict['tensor'] = self_weather_dict['tensor'].type(torch.float32)

self_length = len(self_series)
self_seq_len = seq_len
self_pred_horz = pred_horz

before series: torch.Size([8, 3024])
start_times_ after:  [[2012, 10, 12, 0, 30, 0], [2012, 12, 14, 0, 30, 0], [2013, 2, 15, 0, 30, 0], [2013, 4, 19, 0, 30, 0], [2013, 6, 21, 0, 30, 0], [2013, 8, 23, 0, 30, 0], [2013, 10, 25, 0, 30, 0], [2013, 12, 27, 0, 30, 0]]
start_times after:  [[2012, 10, 12, 0, 30, 0], [2012, 12, 14, 0, 30, 0], [2013, 2, 15, 0, 30, 0], [2013, 4, 19, 0, 30, 0], [2013, 6, 21, 0, 30, 0], [2013, 8, 23, 0, 30, 0], [2013, 10, 25, 0, 30, 0], [2013, 12, 27, 0, 30, 0]]
before concat series: 5561
before concat series: torch.Size([8, 3024])
before normalization series: torch.Size([52713, 3024, 1])
total series shape:  torch.Size([52713, 3024, 1])


In [6]:
dat = self_series[:1334]

print(dat.shape)
# alpha=0.6667

alpha = 0.6
beta = 0.8


train_set = dat[:int(dat.shape[0]*alpha),:,:]
val_set = dat[int(dat.shape[0]*alpha):int(dat.shape[0]*beta),:,:]
test_set = dat[int(dat.shape[0]*beta):,:,:]

print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

torch.Size([1334, 3024, 1])
torch.Size([800, 3024, 1])
torch.Size([267, 3024, 1])
torch.Size([267, 3024, 1])


## Import Models for comparison

In [7]:
#Importing the testing models
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    Prophet,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    VARIMA,
    BATS,
    TBATS,
    StatsForecastAutoARIMA,
    RegressionEnsembleModel,
    RegressionModel,
    Theta,
    FourTheta,
    FFT,
    NBEATSModel,
    TFTModel,
    RNNModel,
    TransformerModel,
)

Importing plotly failed. Interactive plots will not work.


In [8]:
def eval_model(model, train, val, f=None ):
    model.fit(train)
    forecast = model.predict(len(val))
    mseRes = mse(val, forecast)
    rmseRes = rmse(val, forecast)
    # mapeRes = mape(val, forecast)
    maeRes = mae(val, forecast)
    smapeRes = smape(val, forecast)
    # maseRes = mase(val, forecast,train)
    
    # print('model {} obtains MSE: {:.6f}'.format(model,mseRes))
    # print('model {} obtains RMSE: {:.6f}'.format(model,rmseRes))
    # # print('model {} obtains MAPE: {:.6f}'.format(model,mapeRes ))
    # print('model {} obtains MAE: {:.6f}'.format(model, maeRes))
    # print('model {} obtains SMAPE: {:.6f}'.format(model,smapeRes ))
    # print('model {} obtains MASE: {:.6f}'.format(model,maseRes ))

    return forecast, mseRes, rmseRes, maeRes, smapeRes

## Prepare Data

In [9]:
train_set_squeeze = train_set.squeeze(-1)
# print(train_set_squeeze.shape)
train_np = train_set_squeeze.numpy()
# print((type(train_np)))

test_set_squeeze = test_set.squeeze(-1)
# print(train_set_squeeze.shape)
test_np = test_set_squeeze.numpy()

test_np = test_np[:50]

In [10]:
def multi_series_eval(model, series, param_dset_lookback):

    total_MSE = 0.0
    total_RMSE = 0.0
    total_MAE = 0.0
    total_sMAPE = 0.0
    forecast=[]
    
    for i in tqdm(range(len(series)), total=len(series), desc="Series completion", position=1, leave=True):
        train_series = TimeSeries.from_values(series[i])
        train_series = fill_missing_values(train_series)
        train, val = train_series[:param_dset_lookback], train_series[param_dset_lookback:]

        model_forecast, model_MSE, model_RMSE, model_MAE, model_sMAPE = eval_model(model,train,val)
        total_MSE += model_MSE
        total_RMSE += model_RMSE
        total_MAE += model_MAE
        total_sMAPE += model_sMAPE
        forecast.append(model_forecast)

    total_MSE /= len(series)
    total_RMSE /= len(series)
    total_MAE /= len(series)
    total_sMAPE /= len(series)
    total_sMAPE /= 100 # percentage

    with open('stat_model_{}_{}.csv'.format(param_dset_lookback_weeks, param_dset_forecast), 'a', encoding='UTF8') as t:
        writer = csv.writer(t)
        writer.writerow(["London",model,total_MSE,total_RMSE,total_MAE,total_sMAPE,len(val), len(series) ])

    return forecast, total_MSE, total_RMSE, total_MAE, total_sMAPE




In [11]:
RNN_forecast, RNN_MSE, RNN_RMSE, RNN_MAE, RNN_sMAPE =  multi_series_eval(RNNModel(input_chunk_length = 50, 
                                                                              model='RNN', 
                                                                              hidden_dim=25,
                                                                              batch_size=64,
                                                                            n_epochs =20,
                                                                              n_rnn_layers=2, 
                                                                              dropout=0.1, 
                                                                              training_length=50, )
                                                                         ,test_np,param_dset_lookback)

Series completion:   0%|          | 0/50 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 6GB Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 2.0 K 
4 | V             | Linear           | 26    
---------------------------------------------------
2.0 K     Trainable params
0         Non-train

Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                                    | 0/? [00:00<…

...

Predicting: |                                                                                    | 0/? [00:00<…

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 2.0 K 
4 | V             | Linear           | 26    
---------------------------------------------------
2.0 K     Trainable params
0         Non-trainable params
2.0 K     Total params
0.008     Total estimated model params size (MB)


Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                                    | 0/? [00:00<…

In [12]:
FFT_forecast, FFT_MSE, FFT_RMSE, FFT_MAE, FFT_sMAPE =  multi_series_eval(FFT( nr_freqs_to_keep=20,trend= "poly", trend_poly_degree=2),test_np,param_dset_lookback)


Series completion:   0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
# Prophet_forecast, Prophet_MSE, Prophet_RMSE, Prophet_MAE, Prophet_sMAPE = multi_series_eval(Prophet(),test_np,param_dset_lookback)
Arima_forecast, Arima_MSE, Arima_RMSE, Arima_MAE, Arima_sMAPE = multi_series_eval(ARIMA(),test_np,param_dset_lookback)
AArima_forecast, AArima_MSE, AArima_RMSE, AArima_MAE, AArima_sMAPE = multi_series_eval(AutoARIMA(),test_np,param_dset_lookback)
# ExpoSmooth_forecast, ExpoSmooth_MSE, ExpoSmooth_RMSE, ExpoSmooth_MAE, ExpoSmooth_sMAPE = multi_series_eval(ExponentialSmoothing(),test_np,param_dset_lookback)


Series completion:   0%|          | 0/50 [00:00<?, ?it/s]

Series completion:   0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
from darts.utils.utils import SeasonalityMode
Theta_forecast, Theta_MSE, Theta_RMSE, Theta_MAE, Theta_sMAPE = multi_series_eval(Theta(season_mode=SeasonalityMode.ADDITIVE),test_np,param_dset_lookback)
Bats_forecast, Bats_MSE, Bats_RMSE, Bats_MAE, Bats_sMAPE = multi_series_eval(BATS(),test_np,param_dset_lookback)

Series completion:   0%|          | 0/50 [00:00<?, ?it/s]

Series completion:   0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
print(RNN_MSE, RNN_RMSE, RNN_MAE, RNN_sMAPE)
print(FFT_MSE, FFT_RMSE, FFT_MAE, FFT_sMAPE)
print(Arima_MSE, Arima_RMSE, Arima_MAE, Arima_sMAPE)
print(AArima_MSE, AArima_RMSE, AArima_MAE, AArima_sMAPE)
print(Theta_MSE, Theta_RMSE, Theta_MAE, Theta_sMAPE)
print(Bats_MSE, Bats_RMSE, Bats_MAE, Bats_sMAPE)

0.03642344739753753 0.16465652648359538 0.13320636544376613 0.9950199246406555
0.01848237421483475 0.12502120703867384 0.09537515177560596 0.9237392686815453
0.024189073710433946 0.14145724517972652 0.11695404891074444 0.9197205020370421
0.019903297435646082 0.1242500931118254 0.09728342229088177 0.880768345802761
0.030504468965254054 0.1537296572340615 0.12733498942392293 1.0306764340729977
0.015501361086884541 0.11463978936183157 0.08649737683679995 0.8340432339102803


In [16]:
series_num = 1

test_one_series = TimeSeries.from_values(test_np[series_num])
print(test_np[series_num].shape)
test_one_series_actual = test_one_series[param_dset_lookback:]

(3024,)


In [17]:
plt.figure(figsize=(10, 6))
# train.plot(label='Lookback')
test_one_series_actual.plot(label='Actual')
RNN_forecast[series_num].plot(label='RNN Prediction')
FFT_forecast[series_num].plot(label='FFT Prediction')
AArima_forecast[series_num].plot(label='AArima Prediction')
# ExpoSmooth_forecast.plot(label='Expo Prediction')
Theta_forecast[series_num].plot(label='Theta Prediction')
Arima_forecast[series_num].plot(label='Arima Prediction')
Bats_forecast[series_num].plot(label='Bats Prediction')
plt.legend()
